<a href="https://colab.research.google.com/github/Richish/deep_learning_with_python/blob/master/ch64_convnets_for_sequence_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

# Preparing imdb data

In [ ]:
from keras.preprocessing import sequence
from keras.datasets import imdb

Using TensorFlow backend.


In [ ]:
max_features=10_000
max_len=500

(x_train, y_train), (x_test, y_test)=imdb.load_data(num_words=max_features)
print(x_train.shape, x_test.shape)
print("Pad seqiuences")

x_train=sequence.pad_sequences(x_train, maxlen=max_len)
x_test=sequence.pad_sequences(x_test, maxlen=max_len)
print(x_train.shape, x_test.shape)

17465344/17464789 [==============================] - 3s 0us/step
(25000,) (25000,)
Pad seqiuences
(25000, 500) (25000, 500)


# Train and evaluate simple 1d convent on imdb data

In [ ]:
from keras.models import Sequential
from keras.layers import Embedding, Dense, Conv1D, MaxPooling1D, GlobalMaxPool1D
from keras.optimizers import RMSprop

In [ ]:
model=Sequential()
model.add(Embedding(input_dim=max_features, output_dim=128, input_length=max_len))
model.add(Conv1D(filters=32, kernel_size=7, activation='relu'))
model.add(MaxPooling1D(pool_size=5))
model.add(Conv1D(filters=32, kernel_size=7, activation='relu'))
model.add(GlobalMaxPool1D())
model.add(Dense(units=1, activation='sigmoid'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 128)          1280000   
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 494, 32)           28704     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 98, 32)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 92, 32)            7200      
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 1,315,937
Trainable params: 1,315,937
Non-trainable params: 0
____________________________________________

In [ ]:
model.compile(optimizer=RMSprop(learning_rate=1e-4), loss='binary_crossentropy', metrics=['acc'])


In [ ]:
history=model.fit(x=x_train, y=y_train, batch_size=128, epochs=10, validation_split=0.2, use_multiprocessing=True, )

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 9s 444us/step - loss: 0.6927 - acc: 0.5218 - val_loss: 0.6919 - val_acc: 0.5324
Epoch 2/10
20000/20000 [==============================] - 2s 117us/step - loss: 0.6868 - acc: 0.6716 - val_loss: 0.6882 - val_acc: 0.6438
Epoch 3/10
20000/20000 [==============================] - 2s 117us/step - loss: 0.6771 - acc: 0.7540 - val_loss: 0.6755 - val_acc: 0.7242
Epoch 4/10
20000/20000 [==============================] - 2s 117us/step - loss: 0.6480 - acc: 0.7919 - val_loss: 0.6226 - val_acc: 0.7826
Epoch 5/10
20000/20000 [==============================] - 2s 117us/step - loss: 0.5611 - acc: 0.8164 - val_loss: 0.5052 - val_acc: 0.8218
Epoch 6/10
20000/20000 [==============================] - 2s 117us/step - loss: 0.4347 - acc: 0.8473 - val_loss: 0.4000 - val_acc: 0.8424
Epoch 7/10
20000/20000 [==============================] - 2s 116us/step - loss: 0.3456 - acc: 0.8690 - val_loss: 0.3491 - v


# Combination of CNN and RNN on temperature forecasting problem

In [ ]:
!ls
!wget https://s3.amazonaws.com/keras-datasets/jena_climate_2009_2016.csv.zip
!unzip jena_climate_2009_2016.csv.zip


sample_data
--2020-05-23 01:21:25--  https://s3.amazonaws.com/keras-datasets/jena_climate_2009_2016.csv.zip
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.226.251
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.226.251|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13568290 (13M) [application/zip]
Saving to: ‘jena_climate_2009_2016.csv.zip’

jena_climate_2009_2 100%[===================>]  12.94M  5.52MB/s    in 2.3s    

2020-05-23 01:21:29 (5.52 MB/s) - ‘jena_climate_2009_2016.csv.zip’ saved [13568290/13568290]

Archive:  jena_climate_2009_2016.csv.zip
  inflating: jena_climate_2009_2016.csv  


In [ ]:
data_file="jena_climate_2009_2016.csv"

df=pd.read_csv(data_file)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 420551 entries, 0 to 420550
Data columns (total 15 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   Date Time        420551 non-null  object 
 1   p (mbar)         420551 non-null  float64
 2   T (degC)         420551 non-null  float64
 3   Tpot (K)         420551 non-null  float64
 4   Tdew (degC)      420551 non-null  float64
 5   rh (%)           420551 non-null  float64
 6   VPmax (mbar)     420551 non-null  float64
 7   VPact (mbar)     420551 non-null  float64
 8   VPdef (mbar)     420551 non-null  float64
 9   sh (g/kg)        420551 non-null  float64
 10  H2OC (mmol/mol)  420551 non-null  float64
 11  rho (g/m**3)     420551 non-null  float64
 12  wv (m/s)         420551 non-null  float64
 13  max. wv (m/s)    420551 non-null  float64
 14  wd (deg)         420551 non-null  float64
dtypes: float64(14), object(1)
memory usage: 48.1+ MB


In [ ]:
df=df.drop(columns='Date Time')
df.keys()

mean=df.loc[:200_000].mean()
std=df.loc[:200_000].std()
df-=mean
df/=std
df.head()

float_data=df.to_numpy()
float_data.shape

def generator(data, lookback, delay, min_index, max_index, shuffle=False, batch_size=128, step=6):
    """
    data: np.array > of full data set.
    lookback: no. of datapoints to look back in each sample.
    delay: no. of datapoints after current sample for which prediction is to be made, or which corresponds to y
    min_index: consider datapoints starting from this index only
    max_index: Take datapoint upto max this index only
    shuffle: shuffle the data before taking a batch out of data and yielding that
    batch_size: no. of samples to be yielded in a single batch
    step: not all data points will be considered, only datapoints at frequency- 'step' will be considered. 
            since data points aare at 10 min interval so step=6 implies only hourly data will be considered.
    return: a tuple of (samples, data_samples_at_delay). Samples is a 2d np array of shape: (batch_size, lookback, data.shape[-1]). Basically a a batch of samples, 
            where each sample contains a np array of shape(lookback, data.shape[-1])
            data_samples_at_delay is a 2d np array of shape: (batch_size, 1)- 1 since we only need temperature as output
    """
    if max_index is None:
        max_index = len(data) - delay - 1
    i = min_index + lookback # which rows that will contain the end of sample data, delay will be relative to this row.
    while True:
        if shuffle:
            # rows will be equal to batch_size
            rows=np.random.randint(i, max_index, size=batch_size) # basically row indexes.
        else: # not shuffled
            if i+batch_size>max_index: # if reached end, need to rotate and go to beginning to get new batch
                i=min_index+lookback
            rows=np.arange(i, min(i+batch_size,max_index))
            i+=len(rows) # move pointer to where end of next batch would be
        # initializing samples and targets to 0s.
        samples=np.zeros((len(rows), lookback//step, data.shape[-1]))
        targets=np.zeros((len(rows),))
        # setting the value of samples and targets for each sample/target in a batch of batch_size
        for j, row in enumerate(rows):
            indeces=np.arange(row-lookback, row, step)
            samples[j]=data[indeces]
            targets[j]=data[row+delay][1] # we only need the temperature as output/label
        yield samples, targets



In [ ]:
lookback = 24*5*6 # 5 days data
step = 6
delay = 24*6 # 1 day's delay/prediction
batch_size = 128 # training batch size

train_gen = generator(float_data,
                lookback=lookback,
                delay=delay,
                min_index=0,
                max_index=200_000,
                shuffle=True,
                step=step,
                batch_size=batch_size)

val_gen = generator(float_data,
                lookback=lookback,
                delay=delay,
                min_index=200_001,
                max_index=300_000,
                shuffle=True,
                step=step,
                batch_size=batch_size)

test_gen = generator(float_data,
                lookback=lookback,
                delay=delay,
                min_index=300_001,
                max_index=None,
                shuffle=True,
                step=step,
                batch_size=batch_size)
val_steps = (300000 - 200001 - lookback)//128 # how many val steps to look at whole data
test_steps = (len(float_data) - 300001 - lookback)//128 # how many test steps to look at whole data

## Simple 1d convndet on temp forecasting

will not work well since convnets do not have a very good order-sesitivity

In [ ]:
# simple 1d convnet for temp forecasting
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, MaxPooling1D, GlobalMaxPool1D, Dense
from keras.optmimizers import RMSprop


In [ ]:
model=Sequential()
# model.add(Embedding()) embedding not used for timeseries forecasting. Used only for nlp or other sequences where dimensionality of data needs to be changed.
model.add(Conv1D(filters=32, kernel_size=5, activation='relu', input_shape=(None, float_data.shape[-1])))
model.add(MaxPooling1D(pool_size=3))
model.add(Conv1D(filters=32, kernel_size=5, activation='relu'))
model.add(MaxPooling1D(pool_size=3))
model.add(Conv1D(filters=32, kernel_size=5, activation='relu'))
model.add(GlobalMaxPool1D())
model.add(Dense(units=1))
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_4 (Conv1D)            (None, None, 32)          2272      
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, None, 32)          0         
_________________________________________________________________
conv1d_5 (Conv1D)            (None, None, 32)          5152      
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, None, 32)          0         
_________________________________________________________________
conv1d_6 (Conv1D)            (None, None, 32)          5152      
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                

In [ ]:
model.compile(optimizer=RMSprop(learning_rate=1e-4), loss='mae')
history=model.fit_generator(generator=train_gen, steps_per_epoch=500, epochs=20, validation_data=val_gen, validation_steps=val_steps/1000)

Epoch 1/20
500/500 [==============================] - 4s 7ms/step - loss: 0.2632 - val_loss: 0.4040
Epoch 2/20
500/500 [==============================] - 3s 7ms/step - loss: 0.2602 - val_loss: 0.4161
Epoch 3/20
500/500 [==============================] - 3s 7ms/step - loss: 0.2576 - val_loss: 0.4273
Epoch 4/20
500/500 [==============================] - 3s 7ms/step - loss: 0.2561 - val_loss: 0.3994
Epoch 5/20
500/500 [==============================] - 3s 7ms/step - loss: 0.2547 - val_loss: 0.3424
Epoch 6/20
500/500 [==============================] - 3s 7ms/step - loss: 0.2536 - val_loss: 0.4532
Epoch 7/20
500/500 [==============================] - 3s 7ms/step - loss: 0.2499 - val_loss: 0.4343
Epoch 8/20
500/500 [==============================] - 3s 7ms/step - loss: 0.2470 - val_loss: 0.4038
Epoch 9/20
500/500 [==============================] - 3s 7ms/step - loss: 0.2447 - val_loss: 0.3958
Epoch 10/20
500/500 [==============================] - 3s 7ms/step - loss: 0.2436 - val_loss: 0.4070

## Combination of cnn and rnn on tmp data
Use cnn as pre-processing layer to shorten the sequences fed into rnn hence speeding up the rnn training.

Can vary cnn params to adjust the lookback time window.

In [ ]:
# this example uses a higher resolution data(sampled at 3o mins instead of 1 hour) and hence is processing a longer timeseries



### preparing high resolution data generators for jena data


In [ ]:
step = 3 # this is the only thing that changed
lookback = 720
delay = 144
train_gen = generator(float_data,
                          lookback=lookback,
                          delay=delay,
                          min_index=0,
                          max_index=200000,
                          shuffle=True,
                          step=step)
val_gen = generator(float_data,
                      lookback=lookback,
                      delay=delay,
                      min_index=200001,
                      max_index=300000,
                      step=step)
test_gen = generator(float_data,
                        lookback=lookback,
                        delay=delay,
                        min_index=300001,
                        max_index=None,
                        step=step)
val_steps = (300000 - 200001 - lookback) // 128
test_steps = (len(float_data) - 300001 - lookback) // 128

In [ ]:
from keras.models import Sequential
from keras import layers
from keras.optimizers import RMSprop
model = Sequential()
model.add(layers.Conv1D(32, 5, activation='relu',input_shape=(None, float_data.shape[-1])))
model.add(layers.MaxPooling1D(3))
model.add(layers.Conv1D(32, 5, activation='relu'))
model.add(layers.GRU(32, dropout=0.1, recurrent_dropout=0.5))
model.add(layers.Dense(1))
model.summary()
model.compile(optimizer=RMSprop(), loss='mae')

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_7 (Conv1D)            (None, None, 32)          2272      
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, None, 32)          0         
_________________________________________________________________
conv1d_8 (Conv1D)            (None, None, 32)          5152      
_________________________________________________________________
gru_1 (GRU)                  (None, 32)                6240      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 33        
Total params: 13,697
Trainable params: 13,697
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history = model.fit_generator(train_gen,
                                steps_per_epoch=500,
                                epochs=20,
                                validation_data=val_gen,
                                validation_steps=val_steps)

Epoch 1/20
500/500 [==============================] - 115s 231ms/step - loss: 907724271.1197 - val_loss: 0.2699
Epoch 2/20
100/500 [=====>........................] - ETA: 1:21 - loss: 122119411030.1449

In [ ]:
CNN and RNN is faster than RNN alone but IT IS NOT AS GOOD AS RNN IN TERMS OF ACCURACY.
Hence can only be used where accuracy can be compromised for speed.